![Cauldron-logo-JPG-01-800x123.jpg](Cauldron-logo-JPG-01-800x123.jpg)

# Comparison between ILCD CFs and SimaPro and BW2 and EI for EF v3.0

## Changelog

### 1 - (2023-11-17)

+ extract relevant input data
+ merge data from cf listings based on ILCD to EI flow mapping



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm import tqdm

# Extraction

## Extract the mapping ILCE -> EI 3.7

In [3]:
mapping_df = pd.read_excel('ILCD-EFv3.0-ecoinventEFv3.7.xlsx', usecols=["SourceFlowName", "SourceFlowUUID",
                                                                       "TargetFlowName", "TargetFlowUUID"],
                           dtype=str
                          )

In [4]:
mapping_df.tail()

,SourceFlowName,SourceFlowUUID,TargetFlowName,TargetFlowUUID
96756,2-(2-chloroethoxy)ethanol,d7c517de-17ba-4994-9b5f-83cbd7365fc7,NaN,NaN
96757,manganese(2+) oxalate,fa5ce5c9-bf38-40b7-8d68-d069c7f64e79,NaN,NaN
96758,cobalt(2+) carbonate,ec4f4ead-8453-49e3-81f7-5ac6955bb13e,NaN,NaN
96759,methane disulfonic acid disodium salt,d312cb7c-d38e-4c5a-988f-adddfcb78ecf,NaN,NaN
96760,hcfc-124,08a91e70-3ddc-11dd-9f40-0050c2490048,NaN,NaN


## Extract the CFs for EF v3.0 from brightway2 [bw2io 0.8.7]

In [5]:
import bw2data as bd
import bw2io as bi

In [6]:
bd.projects.set_current("ecoinvent-3.8-ei")
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.8-biosphere
	ecoinvent-3.8-cutoff

In [7]:
# Select the ef v3.0 method
ef_30_m = [m for m in bd.methods if m[0] == "EF v3.0"]

cfs = []
# Format the data to make a pandas dataframe
for m in ef_30_m:
    method = bd.Method(m)
    data = method.load()
    for datum in data:
        flow = bd.get_activity(datum[0])
        flow_cat = flow['categories'][0]
        flow_subcat = None
        if len(flow['categories']) == 2:
            flow_subcat = flow['categories'][1]
        entry= {
            "method":m[0],
            "category":m[1],
            "subcategory":m[2],
            "flow": flow['name'],
            "flow_category": flow_cat,
            "flow_subcategory": flow_subcat,
            "flow_uuid": flow['code'],
            "cf": datum[1]
        }
        cfs.append(entry)

In [8]:
cfs_bw2_df = pd.DataFrame(cfs)
cfs_bw2_df

,method,category,subcategory,flow,flow_category,flow_subcategory,flow_uuid,cf
0,EF v3.0,acidification,accumulated exceedance (ae),Ammonia,air,urban air close to ground,9990b51b-7023-4700-bca0-1a32ef921f74,3.02
1,EF v3.0,acidification,accumulated exceedance (ae),Nitrogen oxides,air,lower stratosphere + upper troposphere,4841a0fe-c250-4a39-94a1-1bc31426c0f1,0.74
2,EF v3.0,acidification,accumulated exceedance (ae),Nitrogen oxides,air,None,c1b91234-6f24-417b-8309-46111d09c457,0.74
3,EF v3.0,acidification,accumulated exceedance (ae),Ammonia,air,lower stratosphere + upper troposphere,8494ed3c-0416-4aa5-b100-51a2b2bcadbd,3.02
4,EF v3.0,acidification,accumulated exceedance (ae),Ammonia,air,"low population density, long-term",2b50f643-216a-412b-a0e5-5946867aa2ed,3.02
...,...,...,...,...,...,...,...,...
10499,EF v3.0,water use,user deprivation potential (deprivation-weight...,Water,air,non-urban air or from high stacks,09872080-d143-4fb1-a3a5-647b077107ff,42.95
10500,EF v3.0,water use,user deprivation potential (deprivation-weight...,Water,air,None,075e433b-4be4-448e-9510-9a5029c1ce94,42.95
10501,EF v3.0,water use,user deprivation potential (deprivation-weight...,Water,air,"low population density, long-term",f977a02e-3564-4798-843c-9fb9a18bc18b,42.95
10502,EF v3.0,water use,user deprivation potential (deprivation-weight...,Water,air,lower stratosphere + upper troposphere,f14b59ff-d438-442d-8bad-b53694b8263a,42.95


## Extract ILCD cfs

In [9]:
ilcd_cfs_df = pd.read_excel('EF-LCIAMethod_CF(EF-v3.0).xlsx', sheet_name="lciamethods_CF")

# Comparison

## Resources use, minerals and metals

In [10]:
ilcd_res_df = ilcd_cfs_df.query("LCIAMethod_name == 'Resource use, minerals and metals'")

In [11]:
print(f"There are {len(ilcd_res_df)} CFs in the original ILCD LCIA implementation")

There are 48 CFs in the original ILCD LCIA implementation


In [12]:
ilcd_res_df

,FLOW_uuid,FLOW_name,LCIAMethod_uuid,LCIAMethod_name,LCIAMethod_meanvalue,LCIAMethod_location,FLOW_class0,FLOW_class1,FLOW_class2,LCIAMethod_derivation,LCIAMethod_direction
316243,fe0acd60-3ddc-11dd-ab5f-0050c2490048,aluminium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.090000e-09,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316244,08a91e70-3ddc-11dd-91d5-0050c2490048,antimony,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.000000e+00,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316245,08a91e70-3ddc-11dd-91e4-0050c2490048,arsenic,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",2.970000e-03,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316246,4d9a8790-3ddd-11dd-96cb-0050c2490048,barium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",6.040000e-06,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316247,fe0acd60-3ddc-11dd-a794-0050c2490048,beryllium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.260000e-05,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316248,08a91e70-3ddc-11dd-a0f6-0050c2490048,bismuth,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",4.110000e-02,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316249,fe0acd60-3ddc-11dd-ae95-0050c2490048,boron,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",4.270000e-03,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316250,4d9a8790-3ddd-11dd-8de8-0050c2490048,bromine,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",4.390000e-03,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316251,fe0acd60-3ddc-11dd-a295-0050c2490048,cadmium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.570000e-01,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
316252,fe0acd60-3ddc-11dd-b145-0050c2490048,chlorine,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",2.710000e-05,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT


In [13]:
cfs_bw2_ru = cfs_bw2_df.query("category == 'material resources: metals/minerals'")
# There is only one subcategory:  'abiotic depletion potential (ADP): elements (ultimate reserves)'

In [14]:
print(f"There are {len(cfs_bw2_ru)} CFs in the original ILCD LCIA implementation")

There are 149 CFs in the original ILCD LCIA implementation


In [15]:
cfs_bw2_ru

,method,category,subcategory,flow,flow_category,flow_subcategory,flow_uuid,cf
9934,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Zirconia, as baddeleyite, in ground",natural resource,in ground,e07b4402-abe3-4346-8c42-051c5983bd1e,5.440000e-06
9935,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Tin, 79% in cassiterite, 0.1% in crude ore, in...",natural resource,in ground,31b4eea9-640e-4056-ac2f-0555627af18a,1.620000e-02
9936,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Barium, in ground",natural resource,in ground,240177d8-6f3b-43f5-8d1e-0c18114dfa02,6.040000e-06
9937,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Gold, Au 9.7E-5%, Ag 7.6E-5%, in ore, in ground",natural resource,in ground,c7d38707-3b22-4fb1-b001-0c8cad496a60,5.200000e+01
9938,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Tantalum, in ground",natural resource,in ground,775fdf03-b0bb-5c25-b14d-107231d5b2f0,4.060000e-05
...,...,...,...,...,...,...,...,...
10078,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Platinum, Pt 4.7E-7%, in mixed ore, in ground",natural resource,in ground,68be4a67-89e0-4cfe-a089-fa8706de230e,2.220000e+00
10079,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Pyrite, in ground",natural resource,in ground,c73e75dc-c02d-4192-ab43-faf29c119fae,1.031808e-04
10080,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Gold, Au 7.1E-4%, in ore, in ground",natural resource,in ground,d28f9d42-5df5-41c3-be59-fdfa7ff57112,5.200000e+01
10081,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Iron, 46% in ore, 25% in crude ore, in ground",natural resource,in ground,f77aacc3-2c22-4bda-99ab-fe1110a1b891,5.240000e-08


In [16]:
cfs_bw2_mapped_df = cfs_bw2_ru.merge(mapping_df, left_on="flow_uuid", right_on="TargetFlowUUID")

In [17]:
cfs_bw2_mapped_df

,method,category,subcategory,flow,flow_category,flow_subcategory,flow_uuid,cf,SourceFlowName,SourceFlowUUID,TargetFlowName,TargetFlowUUID
0,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Barium, in ground",natural resource,in ground,240177d8-6f3b-43f5-8d1e-0c18114dfa02,6.040000e-06,barium,4d9a8790-3ddd-11dd-96cb-0050c2490048,"barium, in ground",240177d8-6f3b-43f5-8d1e-0c18114dfa02
1,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Tantalum, in ground",natural resource,in ground,775fdf03-b0bb-5c25-b14d-107231d5b2f0,4.060000e-05,tantalum,1f314b75-6556-11dd-ad8b-0800200c9a66,"tantalum, in ground",775fdf03-b0bb-5c25-b14d-107231d5b2f0
2,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Silicon, in ground",natural resource,in ground,00143719-33a7-5738-aa1b-131f97b4fef3,1.400000e-11,silicon,172a8bf6-6556-11dd-ad8b-0800200c9a66,"silicon, in ground",00143719-33a7-5738-aa1b-131f97b4fef3
3,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Dolomite, in ground",natural resource,in ground,c7aee986-b7d8-4ad9-ad45-1ac0d68e6b78,2.662493e-10,dolomite,fe0acd60-3ddc-11dd-a826-0050c2490048,"dolomite, in ground",c7aee986-b7d8-4ad9-ad45-1ac0d68e6b78
4,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Chrysotile, in ground",natural resource,in ground,c5f5aeb8-7558-4a0c-9594-27621b9cfbc5,5.343573e-10,asbestos (white),fe0acd60-3ddc-11dd-a7ba-0050c2490048,"chrysotile, in ground",c5f5aeb8-7558-4a0c-9594-27621b9cfbc5
...,...,...,...,...,...,...,...,...,...,...,...,...
58,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Colemanite, in ground",natural resource,in ground,ec72c523-9e1a-466a-98c3-e4098e90fd27,6.737185e-04,colemanite,08a91e70-3ddc-11dd-97f2-0050c2490048,"colemanite, in ground",ec72c523-9e1a-466a-98c3-e4098e90fd27
59,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Sodium chloride, in ground",natural resource,in ground,0b9159dd-305d-4add-802f-f7b780ed0289,1.646070e-05,sodium chloride,172ad9fa-6556-11dd-ad8b-0800200c9a66,"sodium chloride, in ground",0b9159dd-305d-4add-802f-f7b780ed0289
60,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Vanadium, in ground",natural resource,in ground,c9c4b80a-73dd-415a-92fb-f877595651c1,7.700000e-07,vanadium,3e4d5086-6556-11dd-ad8b-0800200c9a66,"vanadium, in ground",c9c4b80a-73dd-415a-92fb-f877595651c1
61,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Potassium, in ground",natural resource,in ground,e373f7b4-42e9-4cc7-a73c-f87bec88008b,1.600000e-08,potassium,041ff933-6556-11dd-ad8b-0800200c9a66,"potassium, in ground",e373f7b4-42e9-4cc7-a73c-f87bec88008b


In [18]:
cfs_bw2_ru.to_excel('cfs_bw2_efv30_ru.xlsx')

In [19]:
# The outer join will keep the records from the right (ilcd), even if they don't match flow_uuids from the left (bw/ei)
comp_res_df = cfs_bw2_mapped_df.merge(ilcd_res_df, left_on="SourceFlowUUID", right_on="FLOW_uuid", how="outer")
comp_res_df


,method,category,subcategory,flow,flow_category,flow_subcategory,flow_uuid,cf,SourceFlowName,SourceFlowUUID,...,FLOW_name,LCIAMethod_uuid,LCIAMethod_name,LCIAMethod_meanvalue,LCIAMethod_location,FLOW_class0,FLOW_class1,FLOW_class2,LCIAMethod_derivation,LCIAMethod_direction
0,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Barium, in ground",natural resource,in ground,240177d8-6f3b-43f5-8d1e-0c18114dfa02,6.040000e-06,barium,4d9a8790-3ddd-11dd-96cb-0050c2490048,...,barium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",6.040000e-06,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
1,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Tantalum, in ground",natural resource,in ground,775fdf03-b0bb-5c25-b14d-107231d5b2f0,4.060000e-05,tantalum,1f314b75-6556-11dd-ad8b-0800200c9a66,...,tantalum,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",4.060000e-05,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
2,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Silicon, in ground",natural resource,in ground,00143719-33a7-5738-aa1b-131f97b4fef3,1.400000e-11,silicon,172a8bf6-6556-11dd-ad8b-0800200c9a66,...,silicon,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.400000e-11,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
3,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Dolomite, in ground",natural resource,in ground,c7aee986-b7d8-4ad9-ad45-1ac0d68e6b78,2.662493e-10,dolomite,fe0acd60-3ddc-11dd-a826-0050c2490048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Chrysotile, in ground",natural resource,in ground,c5f5aeb8-7558-4a0c-9594-27621b9cfbc5,5.343573e-10,asbestos (white),fe0acd60-3ddc-11dd-a7ba-0050c2490048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Potassium, in ground",natural resource,in ground,e373f7b4-42e9-4cc7-a73c-f87bec88008b,1.600000e-08,potassium,041ff933-6556-11dd-ad8b-0800200c9a66,...,potassium,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",1.600000e-08,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
62,EF v3.0,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,"Pyrite, in ground",natural resource,in ground,c73e75dc-c02d-4192-ab43-faf29c119fae,1.031808e-04,pyrite,0d7a13f0-6556-11dd-ad8b-0800200c9a66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,bromine,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",4.390000e-03,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,chlorine,b2ad6494-c78d-11e6-9d9d-cec0c932ce01,"Resource use, minerals and metals",2.710000e-05,NaN,Resources,Resources from ground,Non-renewable element resources from ground,UNKNOWN_DERIVATION,INPUT


In [20]:
comp_res_df.to_excel('comp_res_df.xlsx')

### Unmatched flows
From the previous matching, we can see that the EI/Brightway implementation wasn't matched for 3 flows.

In [25]:
comp_res_df[comp_res_df.flow_uuid.isna()]["FLOW_name"]

63     bromine
64    chlorine
65      iodine
Name: FLOW_name, dtype: object

## All methods

In [ ]:
# method categories are named differently
# we need a map to compare them
method_name_mapping = {
    
}